In [1]:
import os

In [2]:
list_file = list(os.walk('./input'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
for file in list_file:
    if '补充' in file and 'Lennon' not in file:
        addition = './input/' + file
    elif 'position' in file:
        position = './input/' + file
    elif 'colorless' in file:
        this = './input/' + file

print('addition:      ' + addition)
print('position:      ' + position)
print('this:          ' + this)

addition:      ./input/补充.xlsx
position:      ./input/position.xlsx
this:          ./input/am-colorless.xlsx


In [3]:
import pandas as pd

In [4]:
df_addition = pd.read_excel(addition,
                            sheet_name=0,
                            header=0,
                            dtype=str).fillna('')[['SKU', '数量']]

df_addition['数量'] = [int(amount.strip()) for amount in df_addition['数量'].tolist()]
df_addition = df_addition.groupby([df_addition['SKU']], as_index=False).sum()

df_addition

,SKU,数量
0,USABS05216J-C,1
1,USABS12770-C,1
2,USABS12780-C,1
3,USABS15540-C,1
4,USABS15830,1
...,...,...
1867,USWSP54521S1-2-YC,1
1868,USWSP54713B1-2-YC,1
1869,USWT35402,1
1870,USWT90720,1


In [5]:
df_position = pd.read_excel(position,
                            header=0,
                            dtype=str).fillna('')

df_position

,SKU,row
0,USAC64980T,1
1,USACC00055,2
2,USACC01207-C,3
3,USACC01311,4
4,USACC01330S-C,5
...,...,...
247326,THB11102H,40890
247327,IC29037Z-C,67653
247328,ALT11462A-FC,78530
247329,CA21685-C,88013


In [6]:
print(df_addition.columns)
print(df_position.columns)

df_addition = pd.merge(left=df_addition,
                       right=df_position,
                       how='left',
                       left_on='SKU',
                       right_on='SKU').fillna('')

df_addition

Index(['SKU', '数量'], dtype='object')
Index(['SKU', 'row'], dtype='object')


,SKU,数量,row
0,USABS05216J-C,1,9350
1,USABS12770-C,1,71473
2,USABS12780-C,1,71478
3,USABS15540-C,1,71489
4,USABS15830,1,
...,...,...,...
1867,USWSP54521S1-2-YC,1,
1868,USWSP54713B1-2-YC,1,
1869,USWT35402,1,4800
1870,USWT90720,1,


In [7]:
df_temp = df_addition.copy()
df_temp['times'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]

df_temp

,SKU,数量,row,times
0,USABS05216J-C,1,9350,1
1,USABS12770-C,1,71473,1
2,USABS12780-C,1,71478,1
3,USABS15540-C,1,71489,1
4,USABS15830,1,,598
...,...,...,...,...
1867,USWSP54521S1-2-YC,1,,598
1868,USWSP54713B1-2-YC,1,,598
1869,USWT35402,1,4800,1
1870,USWT90720,1,,598


In [8]:
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times'] != 1)][['row', '数量']]
df_temp_1 = df_temp_1.groupby([df_temp_1['row']], as_index=False).sum()

df_temp_2 = df_temp[(df_temp['row'] != '') & (df_temp['times']) == 1][['row', '数量']]

df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

df_temp_1

,row,数量
0,9350,1
1,71473,1
2,71478,1
3,71489,1
4,72969,1
...,...,...
1269,22393,1
1270,44719,2
1271,44700,1
1272,68637,1


In [9]:
print(df_addition.columns)
print(df_temp_1.columns)

df_addition = pd.merge(left=df_addition,
                       right=df_temp_1,
                       how='left',
                       left_on='row',
                       right_on='row',
                       suffixes=('_temp', '')).fillna('')

df_addition

Index(['SKU', '数量', 'row'], dtype='object')
Index(['row', '数量'], dtype='object')


,SKU,数量_temp,row,数量
0,USABS05216J-C,1,9350,1.0
1,USABS12770-C,1,71473,1.0
2,USABS12780-C,1,71478,1.0
3,USABS15540-C,1,71489,1.0
4,USABS15830,1,,
...,...,...,...,...
1867,USWSP54521S1-2-YC,1,,
1868,USWSP54713B1-2-YC,1,,
1869,USWT35402,1,4800,1.0
1870,USWT90720,1,,


In [10]:
df_addition['数量'] = [amount_temp if amount == '' else amount for amount_temp, amount in zip(df_addition['数量_temp'].tolist(), df_addition['数量'].tolist())]

df_addition = df_addition[['SKU', '数量']]

df_addition

,SKU,数量
0,USABS05216J-C,1.0
1,USABS12770-C,1.0
2,USABS12780-C,1.0
3,USABS15540-C,1.0
4,USABS15830,1.0
...,...,...
1867,USWSP54521S1-2-YC,1.0
1868,USWSP54713B1-2-YC,1.0
1869,USWT35402,1.0
1870,USWT90720,1.0


In [11]:
df_this = pd.read_excel(this,
                        header=1,
                        usecols='D',
                        dtype=str).fillna('')

df_this

,SKU
0,USVCS11099U
1,USVCS11237
2,USVCS11064U
3,USVCS11070
4,USVCS11516
...,...
13097,USET03566-C
13098,USFP13956-C
13099,USACC11099-C
13100,USTCV16000-2-C


In [12]:
print(df_this.columns)
print(df_addition.columns)

df_this = pd.merge(left=df_this,
                   right=df_addition,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna(0)

df_this

Index(['SKU'], dtype='object')
Index(['SKU', '数量'], dtype='object')


,SKU,数量
0,USVCS11099U,0.0
1,USVCS11237,1.0
2,USVCS11064U,0.0
3,USVCS11070,0.0
4,USVCS11516,0.0
...,...,...
13097,USET03566-C,0.0
13098,USFP13956-C,0.0
13099,USACC11099-C,0.0
13100,USTCV16000-2-C,0.0


In [13]:
import datetime

In [14]:
df_this.to_excel('./补充-' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

print('Done ~')

Done ~
